In [1]:
import yaml
import numpy as np
import os
from math import ceil

from nupic.bindings.math import GetNTAReal
from nupic.engine import Network

from nupic.vision.regions.ImageSensor import ImageSensor

In [2]:
DEFAULT_TM_PARAMS = {
    "columnCount": 1024,
    "inputWidth": 1024,
    "cellsPerColumn": 4,
    'temporalImp' : 'cpp'
}

In [3]:
DEFAULT_SP_PARAMS = {
    "columnCount": 1024,
    "spatialImp": "cpp",
    "inputWidth": 8*8,
    "spVerbosity": 1,
    "synPermConnected": 0.2,
    "synPermActiveInc": 0.0,
    "synPermInactiveDec": 0.0,
    "seed": 1956,
    "numActiveColumnsPerInhArea": 48,
    "globalInhibition": 1,
    "potentialPct": 0.9,
    "boostStrength": 0.0
}

DEFAULT_CLASSIFIER_PARAMS = {
    "distThreshold": 0.000001,
    "maxCategoryCount": 10,
    "cellsPerCol" : 4
}

In [4]:
from math import ceil

class BasePosUpdater:

    def __init__(self, imageSize, step, direction):
        self.imageSize = imageSize
        self.step = step
        if direction == 'horizontal':
            self.idx = 1
        elif direction == 'vertical':
            self.idx = 0
        else:
            raise AttributeError()
        
    def getNextPosition(self, position):
        raise NotImplementedError()
    
    def getImageIterations(self):
        return int(ceil(self.imageSize / float(self.step)))**2

In [5]:
class DiscPosUpdater(BasePosUpdater):
    
    def getNextPosition(self, position):
        offset = position['offset']
        idx = self.idx
        if offset[idx] < self.imageSize - self.step:
            position['offset'][idx] += self.step
        elif offset[1 - idx] < self.imageSize - self.step:
            position['offset'][1 - idx] += self.step
            position['offset'][idx] = 0
        else:
            position['image'] += 1
            position['offset'] = [0, 0]
        return position

In [6]:
class ContPosUpdater(BasePosUpdater):
    
    def __init__(self, *args, **kwargs):
        BasePosUpdater.__init__(self, *args, **kwargs)
        self.straight = True
    
    def getNextPosition(self, position):
        offset = position['offset']
        idx = self.idx
        if self.straight:
            if offset[idx] < self.imageSize - self.step:
                position['offset'][idx] += self.step
            elif offset[1 - idx] < self.imageSize - self.step:
                position['offset'][1 - idx] += self.step
                self.straight = False
            else:
                position['image'] += 1
                position['offset'] = [0, 0]
                self.straight = True
        else:
            if offset[idx] >= self.step:
                position['offset'][idx] -= self.step
            elif offset[1 - idx] < self.imageSize - self.step:
                position['offset'][1 - idx] += self.step
                self.straight = True
            else:
                position['image'] += 1
                position['offset'] = [0, 0]
                self.straight = True
        return position

In [7]:
upd = ContPosUpdater(28, 8, direction='vertical')

pos = {'image' : 0,
      'offset' : [0, 0]}

for i in range(upd.getImageIterations()):
    pos = upd.getNextPosition(pos)
    print(pos)

{'image': 0, 'offset': [8, 0]}
{'image': 0, 'offset': [16, 0]}
{'image': 0, 'offset': [24, 0]}
{'image': 0, 'offset': [24, 8]}
{'image': 0, 'offset': [16, 8]}
{'image': 0, 'offset': [8, 8]}
{'image': 0, 'offset': [0, 8]}
{'image': 0, 'offset': [0, 16]}
{'image': 0, 'offset': [8, 16]}
{'image': 0, 'offset': [16, 16]}
{'image': 0, 'offset': [24, 16]}
{'image': 0, 'offset': [24, 24]}
{'image': 0, 'offset': [16, 24]}
{'image': 0, 'offset': [8, 24]}
{'image': 0, 'offset': [0, 24]}
{'image': 1, 'offset': [0, 0]}


In [8]:
DEFAULT_IMAGESENSOR_PARAMS ={
    "width": 8,
    "height": 8,
    "mode": "bw",
    "background": 0,
    "explorer": yaml.dump(["__main__.myExplorer", {"updater": ContPosUpdater(28, 8, direction='horizontal')}])
}

In [16]:
from nupic.vision.regions.ImageSensorExplorers.BaseExplorer import BaseExplorer
from nupic.regions.sp_region import SPRegion
from nupic.regions.knn_classifier_region import KNNClassifierRegion


class mySensor(ImageSensor):
    
    @classmethod
    def getSpec(cls):
        ns = ImageSensor.getSpec()

        ns['inputs']['feedback'] = {'count': 0,
           'dataType': 'Real32',
           'description': 'Feedback',
           'isDefaultInput': True,
           'regionLevel': False,
           'requireSplitterMap': False,
           'required': True}
        return ns


class myExplorer(BaseExplorer):
    
    def __init__(self, updater, *args, **kwargs):
        BaseExplorer.__init__(self, *args, **kwargs)
        self.updater = updater

    def next(self, seeking=False):
        print self.position['offset'], self.position['image']
        self.position = self.updater.getNextPosition(self.position)
        if self.position['image'] == self.numImages:
            self.position['image'] = 0
            
    def getImageIterations(self):
        return self.updater.getImageIterations()
    
    def first(self, center=True):
        return BaseExplorer.first(self, False)
    
class mySP(SPRegion):
    pass

class myClassifier(KNNClassifierRegion):
    pass

In [17]:
class myNetwork(Network):
    
    def run(self, n):
        tm = self.regions['tm']
        cls = self.regions['classifier']
        sp = self.regions['sp']
        sensor = self.regions['sensor']
        clsLearning = cls.getParameter('learningMode')
        cls.setParameter('learningMode', 0)
        
        for i in range(n):
            print(i)
            if i == n - 1 and clsLearning:
                cls.setParameter('learningMode', 1)
            Network.run(self, 1)
            
            """
            print "SP BUO", sp.getOutputData('bottomUpOut').nonzero()[0][:5]
            print "TM BUI", tm.getInputData('bottomUpIn').nonzero()[0][:5]
            print "TM BUO", tm.getOutputData('bottomUpOut').nonzero()[0][:5]
            print "CLS BUI", cls.getInputData('bottomUpIn').nonzero()[0][:5]
            print "CLS PCAT", cls.getOutputData('categoryProbabilitiesOut')
            print "CLS LRN", cls.getParameter('learningMode')
            print "\n"
            """
            
        
        #sensor = self.regions['sensor']
        #print sensor.getOutputData('resetOut'), sensor.getSelf().explorer[2].position['reset']
        #feedback = sensor.getInputData('feedback')
        #sensor.getSelf().explorer[2].feedback = feedback
        

In [18]:
net = myNetwork()

Network.registerRegion(mySensor)
net.addRegion('sensor', 'py.mySensor', yaml.dump(DEFAULT_IMAGESENSOR_PARAMS))

Network.registerRegion(mySP)
net.addRegion('sp', 'py.mySP', yaml.dump(DEFAULT_SP_PARAMS))

net.addRegion('tm', 'py.TMRegion', yaml.dump(DEFAULT_TM_PARAMS))

Network.registerRegion(myClassifier)
net.addRegion("classifier","py.myClassifier", yaml.dump(DEFAULT_CLASSIFIER_PARAMS))

net.link("sensor", "sp", "UniformLink", "",
           srcOutput = "dataOut", destInput = "bottomUpIn")
net.link("sensor", "sp", "UniformLink", "",
           srcOutput = "resetOut", destInput = "resetIn")
net.link("sensor", "tm", "UniformLink", "",
           srcOutput = "resetOut", destInput = "resetIn")


net.link("sp", "tm", "UniformLink", "",
           srcOutput = "bottomUpOut", destInput = "bottomUpIn")

net.link("tm", "classifier", "UniformLink", "",
           srcOutput = "bottomUpOut", destInput = "bottomUpIn")
net.link("tm", "sensor", "UniformLink", "",
           srcOutput = "bottomUpOut", destInput = "feedback")

net.link("sensor", "classifier", "UniformLink", "",
           srcOutput = "categoryOut", destInput = "categoryIn")

In [19]:
sensor = net.regions["sensor"]
sp = net.regions["sp"]
tm = net.regions['tm']

imgIterations = sensor.getSelf().explorer[2].getImageIterations()

print(imgIterations)

pysp = sp.getSelf()
pytm = tm.getSelf()

classifier = net.regions["classifier"]
dutyCycles = np.zeros(DEFAULT_SP_PARAMS["columnCount"], dtype=GetNTAReal())

dataDir = 'mnist'

sensor.executeCommand(["loadMultipleImages", os.path.join(dataDir, "small_training")])

numTrainingImages = sensor.getParameter("numImages")

print "============= HTM training ================="
classifier.setParameter("inferenceMode", 1)
classifier.setParameter("learningMode", 0)
sp.setParameter("learningMode", 1)
sp.setParameter("inferenceMode", 0)
tm.setParameter("learningMode", 1)
tm.setParameter("inferenceMode", 0)

nTrainingIterations = numTrainingImages
for i in range(nTrainingIterations):
    net.run(imgIterations)
    dutyCycles += pysp._spatialPoolerOutput

        
print "============= Classifier training ================="
classifier.setParameter("inferenceMode", 1)
classifier.setParameter("learningMode", 1)
sp.setParameter("learningMode", 0)
sp.setParameter("inferenceMode", 1)
tm.setParameter("learningMode", 0)
tm.setParameter("inferenceMode", 1)

for i in range(nTrainingIterations):
    net.run(imgIterations)
        
net.save("mnist_net.nta")
tenPct= nTrainingIterations/10
print "My duty cycles:", np.array_str(dutyCycles)
print "Number of nonzero duty cycles:",len(dutyCycles.nonzero()[0])
print "Mean/Max duty cycles:",dutyCycles.mean(), dutyCycles.max()
print "Number of columns that won for > 10% patterns",\
        (dutyCycles>tenPct).sum()
print "Number of columns that won for > 20% patterns",\
        (dutyCycles>2*tenPct).sum()
print "Num categories learned",classifier.getParameter("categoryCount")
print "Number of patterns stored",classifier.getParameter("patternCount")


16
============= HTM training =================
0
[0, 0] 0
1
[0, 8] 0
2
[0, 16] 0
3
[0, 24] 0
4
[8, 24] 0
5
[8, 16] 0
6
[8, 8] 0
7
[8, 0] 0
8
[16, 0] 0
9
[16, 8] 0
10
[16, 16] 0
11
[16, 24] 0
12
[24, 24] 0
13
[24, 16] 0
14
[24, 8] 0
15
[24, 0] 0
0
[0, 0] 1
1
[0, 8] 1
2
[0, 16] 1
3
[0, 24] 1
4
[8, 24] 1
5
[8, 16] 1
6
[8, 8] 1
7
[8, 0] 1
8
[16, 0] 1
9
[16, 8] 1
10
[16, 16] 1
11
[16, 24] 1
12
[24, 24] 1
13
[24, 16] 1
14
[24, 8] 1
15
[24, 0] 1
0
[0, 0] 2
1
[0, 8] 2
2
[0, 16] 2
3
[0, 24] 2
4
[8, 24] 2
5
[8, 16] 2
6
[8, 8] 2
7
[8, 0] 2
8
[16, 0] 2
9
[16, 8] 2
10
[16, 16] 2
11
[16, 24] 2
12
[24, 24] 2
13
[24, 16] 2
14
[24, 8] 2
15
[24, 0] 2
0
[0, 0] 3
1
[0, 8] 3
2
[0, 16] 3
3
[0, 24] 3
4
[8, 24] 3
5
[8, 16] 3
6
[8, 8] 3
7
[8, 0] 3
8
[16, 0] 3
9
[16, 8] 3
10
[16, 16] 3
11
[16, 24] 3
12
[24, 24] 3
13
[24, 16] 3
14
[24, 8] 3
15
[24, 0] 3
0
[0, 0] 4
1
[0, 8] 4
2
[0, 16] 4
3
[0, 24] 4
4
[8, 24] 4
5
[8, 16] 4
6
[8, 8] 4
7
[8, 0] 4
8
[16, 0] 4
9
[16, 8] 4
10
[16, 16] 4
11
[16, 24] 4
12
[24, 24] 4
13

10
[16, 16] 40
11
[16, 24] 40
12
[24, 24] 40
13
[24, 16] 40
14
[24, 8] 40
15
[24, 0] 40
0
[0, 0] 41
1
[0, 8] 41
2
[0, 16] 41
3
[0, 24] 41
4
[8, 24] 41
5
[8, 16] 41
6
[8, 8] 41
7
[8, 0] 41
8
[16, 0] 41
9
[16, 8] 41
10
[16, 16] 41
11
[16, 24] 41
12
[24, 24] 41
13
[24, 16] 41
14
[24, 8] 41
15
[24, 0] 41
0
[0, 0] 42
1
[0, 8] 42
2
[0, 16] 42
3
[0, 24] 42
4
[8, 24] 42
5
[8, 16] 42
6
[8, 8] 42
7
[8, 0] 42
8
[16, 0] 42
9
[16, 8] 42
10
[16, 16] 42
11
[16, 24] 42
12
[24, 24] 42
13
[24, 16] 42
14
[24, 8] 42
15
[24, 0] 42
0
[0, 0] 43
1
[0, 8] 43
2
[0, 16] 43
3
[0, 24] 43
4
[8, 24] 43
5
[8, 16] 43
6
[8, 8] 43
7
[8, 0] 43
8
[16, 0] 43
9
[16, 8] 43
10
[16, 16] 43
11
[16, 24] 43
12
[24, 24] 43
13
[24, 16] 43
14
[24, 8] 43
15
[24, 0] 43
0
[0, 0] 44
1
[0, 8] 44
2
[0, 16] 44
3
[0, 24] 44
4
[8, 24] 44
5
[8, 16] 44
6
[8, 8] 44
7
[8, 0] 44
8
[16, 0] 44
9
[16, 8] 44
10
[16, 16] 44
11
[16, 24] 44
12
[24, 24] 44
13
[24, 16] 44
14
[24, 8] 44
15
[24, 0] 44
0
[0, 0] 45
1
[0, 8] 45
2
[0, 16] 45
3
[0, 24] 45
4
[8, 

7
[8, 0] 79
8
[16, 0] 79
9
[16, 8] 79
10
[16, 16] 79
11
[16, 24] 79
12
[24, 24] 79
13
[24, 16] 79
14
[24, 8] 79
15
[24, 0] 79
0
[0, 0] 80
1
[0, 8] 80
2
[0, 16] 80
3
[0, 24] 80
4
[8, 24] 80
5
[8, 16] 80
6
[8, 8] 80
7
[8, 0] 80
8
[16, 0] 80
9
[16, 8] 80
10
[16, 16] 80
11
[16, 24] 80
12
[24, 24] 80
13
[24, 16] 80
14
[24, 8] 80
15
[24, 0] 80
0
[0, 0] 81
1
[0, 8] 81
2
[0, 16] 81
3
[0, 24] 81
4
[8, 24] 81
5
[8, 16] 81
6
[8, 8] 81
7
[8, 0] 81
8
[16, 0] 81
9
[16, 8] 81
10
[16, 16] 81
11
[16, 24] 81
12
[24, 24] 81
13
[24, 16] 81
14
[24, 8] 81
15
[24, 0] 81
0
[0, 0] 82
1
[0, 8] 82
2
[0, 16] 82
3
[0, 24] 82
4
[8, 24] 82
5
[8, 16] 82
6
[8, 8] 82
7
[8, 0] 82
8
[16, 0] 82
9
[16, 8] 82
10
[16, 16] 82
11
[16, 24] 82
12
[24, 24] 82
13
[24, 16] 82
14
[24, 8] 82
15
[24, 0] 82
0
[0, 0] 83
1
[0, 8] 83
2
[0, 16] 83
3
[0, 24] 83
4
[8, 24] 83
5
[8, 16] 83
6
[8, 8] 83
7
[8, 0] 83
8
[16, 0] 83
9
[16, 8] 83
10
[16, 16] 83
11
[16, 24] 83
12
[24, 24] 83
13
[24, 16] 83
14
[24, 8] 83
15
[24, 0] 83
0
[0, 0] 84
1
[0, 

10
[16, 16] 118
11
[16, 24] 118
12
[24, 24] 118
13
[24, 16] 118
14
[24, 8] 118
15
[24, 0] 118
0
[0, 0] 119
1
[0, 8] 119
2
[0, 16] 119
3
[0, 24] 119
4
[8, 24] 119
5
[8, 16] 119
6
[8, 8] 119
7
[8, 0] 119
8
[16, 0] 119
9
[16, 8] 119
10
[16, 16] 119
11
[16, 24] 119
12
[24, 24] 119
13
[24, 16] 119
14
[24, 8] 119
15
[24, 0] 119
0
[0, 0] 120
1
[0, 8] 120
2
[0, 16] 120
3
[0, 24] 120
4
[8, 24] 120
5
[8, 16] 120
6
[8, 8] 120
7
[8, 0] 120
8
[16, 0] 120
9
[16, 8] 120
10
[16, 16] 120
11
[16, 24] 120
12
[24, 24] 120
13
[24, 16] 120
14
[24, 8] 120
15
[24, 0] 120
0
[0, 0] 121
1
[0, 8] 121
2
[0, 16] 121
3
[0, 24] 121
4
[8, 24] 121
5
[8, 16] 121
6
[8, 8] 121
7
[8, 0] 121
8
[16, 0] 121
9
[16, 8] 121
10
[16, 16] 121
11
[16, 24] 121
12
[24, 24] 121
13
[24, 16] 121
14
[24, 8] 121
15
[24, 0] 121
0
[0, 0] 122
1
[0, 8] 122
2
[0, 16] 122
3
[0, 24] 122
4
[8, 24] 122
5
[8, 16] 122
6
[8, 8] 122
7
[8, 0] 122
8
[16, 0] 122
9
[16, 8] 122
10
[16, 16] 122
11
[16, 24] 122
12
[24, 24] 122
13
[24, 16] 122
14
[24, 8] 122
1

12
[24, 24] 156
13
[24, 16] 156
14
[24, 8] 156
15
[24, 0] 156
0
[0, 0] 157
1
[0, 8] 157
2
[0, 16] 157
3
[0, 24] 157
4
[8, 24] 157
5
[8, 16] 157
6
[8, 8] 157
7
[8, 0] 157
8
[16, 0] 157
9
[16, 8] 157
10
[16, 16] 157
11
[16, 24] 157
12
[24, 24] 157
13
[24, 16] 157
14
[24, 8] 157
15
[24, 0] 157
0
[0, 0] 158
1
[0, 8] 158
2
[0, 16] 158
3
[0, 24] 158
4
[8, 24] 158
5
[8, 16] 158
6
[8, 8] 158
7
[8, 0] 158
8
[16, 0] 158
9
[16, 8] 158
10
[16, 16] 158
11
[16, 24] 158
12
[24, 24] 158
13
[24, 16] 158
14
[24, 8] 158
15
[24, 0] 158
0
[0, 0] 159
1
[0, 8] 159
2
[0, 16] 159
3
[0, 24] 159
4
[8, 24] 159
5
[8, 16] 159
6
[8, 8] 159
7
[8, 0] 159
8
[16, 0] 159
9
[16, 8] 159
10
[16, 16] 159
11
[16, 24] 159
12
[24, 24] 159
13
[24, 16] 159
14
[24, 8] 159
15
[24, 0] 159
0
[0, 0] 160
1
[0, 8] 160
2
[0, 16] 160
3
[0, 24] 160
4
[8, 24] 160
5
[8, 16] 160
6
[8, 8] 160
7
[8, 0] 160
8
[16, 0] 160
9
[16, 8] 160
10
[16, 16] 160
11
[16, 24] 160
12
[24, 24] 160
13
[24, 16] 160
14
[24, 8] 160
15
[24, 0] 160
0
[0, 0] 161
1
[0,

9
[16, 8] 193
10
[16, 16] 193
11
[16, 24] 193
12
[24, 24] 193
13
[24, 16] 193
14
[24, 8] 193
15
[24, 0] 193
0
[0, 0] 194
1
[0, 8] 194
2
[0, 16] 194
3
[0, 24] 194
4
[8, 24] 194
5
[8, 16] 194
6
[8, 8] 194
7
[8, 0] 194
8
[16, 0] 194
9
[16, 8] 194
10
[16, 16] 194
11
[16, 24] 194
12
[24, 24] 194
13
[24, 16] 194
14
[24, 8] 194
15
[24, 0] 194
0
[0, 0] 195
1
[0, 8] 195
2
[0, 16] 195
3
[0, 24] 195
4
[8, 24] 195
5
[8, 16] 195
6
[8, 8] 195
7
[8, 0] 195
8
[16, 0] 195
9
[16, 8] 195
10
[16, 16] 195
11
[16, 24] 195
12
[24, 24] 195
13
[24, 16] 195
14
[24, 8] 195
15
[24, 0] 195
0
[0, 0] 196
1
[0, 8] 196
2
[0, 16] 196
3
[0, 24] 196
4
[8, 24] 196
5
[8, 16] 196
6
[8, 8] 196
7
[8, 0] 196
8
[16, 0] 196
9
[16, 8] 196
10
[16, 16] 196
11
[16, 24] 196
12
[24, 24] 196
13
[24, 16] 196
14
[24, 8] 196
15
[24, 0] 196
0
[0, 0] 197
1
[0, 8] 197
2
[0, 16] 197
3
[0, 24] 197
4
[8, 24] 197
5
[8, 16] 197
6
[8, 8] 197
7
[8, 0] 197
8
[16, 0] 197
9
[16, 8] 197
10
[16, 16] 197
11
[16, 24] 197
12
[24, 24] 197
13
[24, 16] 197
14

11
[16, 24] 229
12
[24, 24] 229
13
[24, 16] 229
14
[24, 8] 229
15
[24, 0] 229
0
[0, 0] 230
1
[0, 8] 230
2
[0, 16] 230
3
[0, 24] 230
4
[8, 24] 230
5
[8, 16] 230
6
[8, 8] 230
7
[8, 0] 230
8
[16, 0] 230
9
[16, 8] 230
10
[16, 16] 230
11
[16, 24] 230
12
[24, 24] 230
13
[24, 16] 230
14
[24, 8] 230
15
[24, 0] 230
0
[0, 0] 231
1
[0, 8] 231
2
[0, 16] 231
3
[0, 24] 231
4
[8, 24] 231
5
[8, 16] 231
6
[8, 8] 231
7
[8, 0] 231
8
[16, 0] 231
9
[16, 8] 231
10
[16, 16] 231
11
[16, 24] 231
12
[24, 24] 231
13
[24, 16] 231
14
[24, 8] 231
15
[24, 0] 231
0
[0, 0] 232
1
[0, 8] 232
2
[0, 16] 232
3
[0, 24] 232
4
[8, 24] 232
5
[8, 16] 232
6
[8, 8] 232
7
[8, 0] 232
8
[16, 0] 232
9
[16, 8] 232
10
[16, 16] 232
11
[16, 24] 232
12
[24, 24] 232
13
[24, 16] 232
14
[24, 8] 232
15
[24, 0] 232
0
[0, 0] 233
1
[0, 8] 233
2
[0, 16] 233
3
[0, 24] 233
4
[8, 24] 233
5
[8, 16] 233
6
[8, 8] 233
7
[8, 0] 233
8
[16, 0] 233
9
[16, 8] 233
10
[16, 16] 233
11
[16, 24] 233
12
[24, 24] 233
13
[24, 16] 233
14
[24, 8] 233
15
[24, 0] 233
0


9
[16, 8] 265
10
[16, 16] 265
11
[16, 24] 265
12
[24, 24] 265
13
[24, 16] 265
14
[24, 8] 265
15
[24, 0] 265
0
[0, 0] 266
1
[0, 8] 266
2
[0, 16] 266
3
[0, 24] 266
4
[8, 24] 266
5
[8, 16] 266
6
[8, 8] 266
7
[8, 0] 266
8
[16, 0] 266
9
[16, 8] 266
10
[16, 16] 266
11
[16, 24] 266
12
[24, 24] 266
13
[24, 16] 266
14
[24, 8] 266
15
[24, 0] 266
0
[0, 0] 267
1
[0, 8] 267
2
[0, 16] 267
3
[0, 24] 267
4
[8, 24] 267
5
[8, 16] 267
6
[8, 8] 267
7
[8, 0] 267
8
[16, 0] 267
9
[16, 8] 267
10
[16, 16] 267
11
[16, 24] 267
12
[24, 24] 267
13
[24, 16] 267
14
[24, 8] 267
15
[24, 0] 267
0
[0, 0] 268
1
[0, 8] 268
2
[0, 16] 268
3
[0, 24] 268
4
[8, 24] 268
5
[8, 16] 268
6
[8, 8] 268
7
[8, 0] 268
8
[16, 0] 268
9
[16, 8] 268
10
[16, 16] 268
11
[16, 24] 268
12
[24, 24] 268
13
[24, 16] 268
14
[24, 8] 268
15
[24, 0] 268
0
[0, 0] 269
1
[0, 8] 269
2
[0, 16] 269
3
[0, 24] 269
4
[8, 24] 269
5
[8, 16] 269
6
[8, 8] 269
7
[8, 0] 269
8
[16, 0] 269
9
[16, 8] 269
10
[16, 16] 269
11
[16, 24] 269
12
[24, 24] 269
13
[24, 16] 269
14

RuntimeError: Python exception raised. Unable to extract info

In [15]:
print "Reading test images"
sensor.executeCommand(["loadMultipleImages", 'mnist/testing'])
numTestImages = sensor.getParameter("numImages")
print "Number of test images",numTestImages
imgIterations = sensor.getSelf().explorer[2].getImageIterations()

classifier.setParameter("inferenceMode", 1)
classifier.setParameter("learningMode", 0)
sp.setParameter("inferenceMode", 1)
sp.setParameter("learningMode", 0)
tm.setParameter("inferenceMode", 1)
tm.setParameter("learningMode", 0)

numCorrect = 0
for i in range(numTestImages):
    net.run(imgIterations)
    inferredCategory = classifier.getOutputData("categoriesOut").argmax()
    if sensor.getOutputData("categoryOut") == inferredCategory:
        numCorrect += 1
    if i%(numTestImages/10)== 0:
        print "Iteration",i,"numCorrect=",numCorrect

# Some interesting statistics
print "Number of test images",numTestImages
print "num correct=",numCorrect
print "pct correct=",(100.0*numCorrect) / numTestImages

Reading test images
Number of test images 10000
Iteration 0 numCorrect= 0
Iteration 1000 numCorrect= 67
Iteration 2000 numCorrect= 514
Iteration 3000 numCorrect= 568
Iteration 4000 numCorrect= 695
Iteration 5000 numCorrect= 739
Iteration 6000 numCorrect= 783
Iteration 7000 numCorrect= 803
Iteration 8000 numCorrect= 843
Iteration 9000 numCorrect= 1139
Number of test images 10000
num correct= 1167
pct correct= 11.67


# Default:

In [9]:
print "Reading test images"
sensor.executeCommand(["loadMultipleImages", 'mnist/testing'])
numTestImages = sensor.getParameter("numImages")
print "Number of test images",numTestImages

sensor.setParameter("explorer", yaml.dump(["__main__.myExplorer",
                                         {"replacement": False}]))
classifier.setParameter("inferenceMode", 1)
classifier.setParameter("learningMode", 0)
sp.setParameter("inferenceMode", 1)
sp.setParameter("learningMode", 0)

numCorrect = 0
for i in range(numTestImages):
    net.run(1)
    inferredCategory = classifier.getOutputData("categoriesOut").argmax()
    if sensor.getOutputData("categoryOut") == inferredCategory:
        numCorrect += 1
    if i%(numTestImages/10)== 0:
        print "Iteration",i,"numCorrect=",numCorrect

# Some interesting statistics
print "Number of test images",numTestImages
print "num correct=",numCorrect
print "pct correct=",(100.0*numCorrect) / numTestImages

Reading test images
Number of test images 10000
Iteration 0 numCorrect= 1
Iteration 1000 numCorrect= 843
Iteration 2000 numCorrect= 1686
Iteration 3000 numCorrect= 2530
Iteration 4000 numCorrect= 3375
Iteration 5000 numCorrect= 4202
Iteration 6000 numCorrect= 5060
Iteration 7000 numCorrect= 5903
Iteration 8000 numCorrect= 6755
Iteration 9000 numCorrect= 7614
Number of test images 10000
num correct= 8464
pct correct= 84.64
